In [1]:
import cv2
import os
import numpy as np

In [2]:

def chop_middle(original_picfolder_path, target_folder_path, chop_size, mode=1, stride=14):
    '''截取图片部分''' 
    # original_picfolder_path = 'D:\\data\\archive\\Set14\\Set14' # 原始图片所在文件夹
    # target_folder_path = 'D:\data\archive\Set14\set14_hr128' # 截取好的图片放置文件夹
    # chop_size = 128 # 截取部分大小

    imgs = os.listdir(original_picfolder_path)
    imglist = [os.path.join(original_picfolder_path, x) for x in imgs]
    index = 0
    chop_size_m = chop_size // 2

    for img in imglist:
        img = cv2.imread(img)
        height, weight = img.shape[:2]
        
        if mode == 1: # 截取图片中间
            height_m, weight_m = height // 2, weight // 2

            if height_m >= chop_size_m and weight_m >= chop_size_m:
                name = target_folder_path + '\\img' + str(index) + '.png'
                chop = img[(height_m-chop_size_m):(height_m+chop_size_m), (weight_m-chop_size_m):(weight_m+chop_size_m)]
                cv2.imwrite(name, chop)
                index += 1
                
        if mode == 2: # 截图所有
            h_num = height // chop_size
            w_num = weight // chop_size
            
            if h_num and w_num:
                for i in range(h_num):
                    for j in range(w_num):
                        chop = img[i*chop_size:(i+1)*chop_size, j*chop_size:(j+1)*chop_size]
                        name = target_folder_path + '\\img' + str(index) + '.png'
                        cv2.imwrite(name, chop)
                        index += 1
        if mode == 3: # 剪掉边边
            chop = img[6:height-6, 6:weight-6]
            name = target_folder_path + '\\img' + str(index) + '.png'
            cv2.imwrite(name, chop)
            index += 1
            
        if mode == 4:
            h_end = chop_size
            h_begin = 0
            
            while h_end<=height:
                w_begin = 0
                w_end = chop_size
                while w_end<=weight:
                    chop = img[h_begin:h_end, w_begin:w_end]
                    name = target_folder_path + '\\img' + str(index) + '.png'
                    cv2.imwrite(name, chop)
                    
                    w_begin += stride
                    w_end += stride
                    index += 1
                h_begin += stride
                h_end += stride

In [3]:
def ext(pv):
    if pv > 255:
        return 255
    if pv < 0:
        return 0
    else:
        return pv

# 高斯噪点的生成
def gauss_noise(image):
    h, w, ch = image.shape
    for row in range(h):
        for col in range(w):
        
        	# numpy.random.normal(loc, scale, size)生成高斯分布的概率密度随机数
        	# loc：float代表生成的高斯分布的随机数的均值
        	# scale：float 代表这个分布的方差
        	# size：int or tuple of ints 输出的shape，默认为None，只输出一个值 
        	# 当指定整数时，输出整数个值，也可以输出（a, b）→ a 行 b 列
            s = np.random.normal(0, 10, 3)
            # 去除每一个像素的三个通道值
            b = image[row, col, 0]
            g = image[row, col, 1]
            r = image[row, col, 2]
            # 在每一个像素的三个通道值上加上高斯噪声
            image[row, col, 0] = ext(b + s[0])
            image[row, col, 1] = ext(g + s[1])
            image[row, col, 2] = ext(r + s[2])

In [4]:
def make_lr(original_picfolder_path, target_folder_path, times=2):
    '''使用bicuic生成lr图片'''
    
    imgs = os.listdir(original_picfolder_path)
    imglist = [os.path.join(original_picfolder_path, x) for x in imgs]
    index = 0
    for path in imglist:
        img = cv2.imread(path)
        # gauss_noise(img)
        h, w = img.shape[:2]
        img_lr = cv2.resize(img, (w//times, h//times))
        img_l2h = cv2.resize(img_lr, (w, h), interpolation=cv2.INTER_CUBIC)
        name = target_folder_path + '\\img' + str(index) + '.png'
        cv2.imwrite(name, img_l2h)
        index += 1

In [5]:
oripath = 'D:\\data\\archive\\T91'
hr_path = 'D:\\data\\archive\\train\\t91'
hr6_path = 'D:\\data\\archive\\train\\t916'
lr_path = 'D:\\data\\archive\\train\\tl'

chop_middle(oripath, hr_path, 32, mode=4)
chop_middle(hr_path, hr6_path, 32, mode=3)
make_lr(hr_path, lr_path, times=2)


In [19]:
chop_middle('D:\\data\\archive\\Set14\\set14_lr128', 'D:\\data\\archive\\Set14\\compare',chop_size=128, mode=3)

In [20]:
make_lr('D:\\data\\archive\\Set14\\Set14', 'D:\\data\\archive\\Set14\\set14_lr128', times=2)

In [21]:
make_lr('D:\\data\\archive\\train\\t91', 'D:\\data\\archive\\train\\tl', times=2)